In [49]:
import torch
from text import text_to_sequence, cmudict
from text.symbols import symbols, _letters, _pad, _punctuation
from text import cleaners
import commons
import attentions
import modules
import models
import model_vad
import utils
import random
import os
from unidecode import unidecode
import re

In [52]:
letter_new = 'AEINOQUabdefghijkmnoprstuvwyzʃʧʦ↓↑ ' + 'ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㄲㄸㅃㅆㅉㅏㅓㅗㅜㅡㅣㅐㅔ '
_newletters = [s for s in letter_new if s not in _letters]
"".join(_newletters)

'ʃʧʦ↓↑ ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㄲㄸㅃㅆㅉㅏㅓㅗㅜㅡㅣㅐㅔ '

In [55]:
cleaners.japanese_cleaners("このジェシー役でステイモスは、エミー賞にノミネートされたこともある。")

'ko↑no je↑ʃii↓yakude sU↑tei↓mosuwa, e↑mii↓ʃooni no↑mine↓eto sa↑reta ko↑to↓mo a↓ru.'

In [42]:
cleaners.korean_cleaners("계속해서 저렇게 억지 주장을 하고 있으니 저 모양이지.")

'ㄱㅣㅔㅅㅗㄱㅎㅐㅅㅓ ㅈㅓㄹㅓㅎㄱㅔ ㅇㅓㄱㅈㅣ ㅈㅜㅈㅏㅇㅇㅡㄹ ㅎㅏㄱㅗ ㅇㅣㅆㅇㅡㄴㅣ ㅈㅓ ㅁㅗㅇㅣㅏㅇㅇㅣㅈㅣ.'

In [5]:
len("ㄱㅣㅔㅅㅗㄱㅎㅐㅅㅓ ㅈㅓㄹㅓㅎㄱㅔ ㅇㅓㄱㅈㅣ ㅈㅜㅈㅏㅇㅇㅡㄹ ㅎㅏㄱㅗ ㅇㅣㅆㅇㅡㄴㅣ ㅈㅓ ㅁㅗㅇㅣㅏㅇㅇㅣㅈㅣ.")

61

In [6]:
len("계속해서 저렇게 억지 주장을 하고 있으니 저 모양이지.")

30

In [2]:
device = torch.device('cpu')

In [6]:
hps = utils.get_hparams_from_file("configs/base_blank_emo_lang.json")

generator = models.FlowGenerator(
        n_vocab=len(symbols) + getattr(hps.data, "add_blank", False),
        out_channels=hps.data.n_mel_channels,
        n_lang=hps.data.n_lang,
        # n_speakers=hps.data.n_speakers,
        **hps.model
    ).to(device)

Use Speaker Embed Linear Norm
Use Multilanguage Cathegorical


In [9]:
checkpoint_path = hps.train.warm_start_checkpoint
model = generator
ignore_layers = []

In [108]:
assert os.path.isfile(checkpoint_path)
print("Warm starting model from checkpoint '{}'".format(checkpoint_path))
checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
model_dict = checkpoint_dict['model']

Warm starting model from checkpoint 'logs/base_blank_emo_lang_onlyintextencoder/G_143.pth'


In [181]:
def transfer_weight(original_tensor, target_size):
    differences = [target_size[i] - original_tensor.size(i) for i in range(len(target_size))]
    for i, diff in enumerate(differences):
        if diff > 0:
            new_dims = list(original_tensor.size())
            new_dims[i] = diff
            rand_weight = torch.randn(*new_dims)
            original_tensor = torch.cat([original_tensor, rand_weight], dim=i)
        elif diff < 0:
            slices = []
            for j in range(len(target_size)):
                if j == i:
                    slices.append(slice(0, original_tensor.size(j) + diff))
                else:
                    slices.append(slice(0, original_tensor.size(j)))
            slices[i] = slice(0, target_size[i])
            original_tensor = original_tensor[slices]

    return original_tensor

In [182]:
mismatched_layers = []
for key, value in model_dict.items(): # model_dict warmstart weight
    if hasattr(model, 'module'): # model is current model
        if key in model.module.state_dict() and value.size() != model.module.state_dict()[key].size():
            try:
                model_dict[key] = transfer_weight(model_dict[key], model.module.state_dict()[key].size())
            except:
                mismatched_layers.append(key)
    else:
        if key in model.state_dict() and value.size() != model.state_dict()[key].size():
            try:
                model_dict[key] = transfer_weight(model_dict[key], model.state_dict()[key].size())
            except:
                mismatched_layers.append(key)

In [138]:
original_tensor.shape

torch.Size([390, 390, 5])

In [134]:
new_dims

[198, 192, 5]

In [132]:
new_dims = list(original_tensor.size())

In [ ]:
rand_weight = torch.randn(*new_dims)

In [129]:
diff

6

In [127]:
differences

[6, 6, 0]

In [126]:
target_size

torch.Size([198, 198, 5])

In [122]:
original_tensor.size()

torch.Size([192, 192, 5])

In [121]:
new_dims

[192, 192, 5, 6]

In [120]:
rand_weight.shape

torch.Size([192, 192, 5, 6])

In [117]:
def pad_tensor_to_match(original_tensor, target_size):
    differences = [target_size[i] - original_tensor.size(i) for i in range(len(target_size))]

    # Pad along each dimension
    for i, diff in enumerate(differences):
        if diff > 0:
            new_dims = list(original_tensor.size())
            new_dims.append(diff)
            rand_weight = torch.randn(*new_dims)
            original_tensor = torch.cat([original_tensor, rand_weight], dim=i)
        elif diff < 0:
            slices = [slice(0, original_tensor.size(j)) for j in range(len(target_size))]
            slices[i] = slice(0, target_size[i])
            original_tensor = original_tensor[slices]

    return original_tensor

In [118]:
pad_tensor_to_match(model_dict[key], model.state_dict()[key].size())

RuntimeError: Tensors must have same number of dimensions: got 3 and 4

In [ ]:
    size_diff = [max(0, new_size[i] - original_tensor.size(i)) for i in range(len(original_tensor.size()))]

    # Create new tensor with random values
    new_tensor = torch.randn(*new_size)

    # Index to slice the original tensor based on its dimensions
    slices = [slice(0, original_tensor.size(i)) for i in range(len(original_tensor.size()))]

    # Update slices based on the size difference
    for i in range(len(size_diff)):
        slices[i] = slice(0, original_tensor.size(i))

    # Copy values from the original tensor to the corresponding slice in the new tensor
    new_tensor[slices] = original_tensor

In [105]:
model.state_dict()[key].size()

torch.Size([198])

In [104]:
difference_0

6

In [100]:
model_dict[key].size()

torch.Size([192])

In [101]:
value.size()

torch.Size([192])

In [91]:
model_dict[key][:,0:2,:].shape

torch.Size([198, 2, 5])

In [85]:
model.state_dict()[key].size()[0] - value.size()[0]

6

In [62]:
model_dict[key].size()

torch.Size([192])

In [58]:
model.state_dict()[key].weight

AttributeError: 'Tensor' object has no attribute 'weight'

In [52]:
model_dict[key] = torch.cat([model_dict[key], new_row], axis=0)

In [53]:
new_row = torch.randn(model_dict[key].size()[-3], difference_1, model_dict[key].size()[-1])

In [54]:
new_row.shape

torch.Size([198, 6, 5])

In [55]:
model_dict[key] = torch.cat([model_dict[key], new_row], axis=1)

In [56]:
model_dict[key].shape

torch.Size([198, 198, 5])

In [37]:
new_row.shape

torch.Size([6, 198, 5])

In [ ]:
# Original tensor size
original_size = (192, 192, 5)

# Desired size for the new tensor
new_size = (198, 198, 5)

# Calculate the number of new rows to add
num_new_rows = new_size[0] - original_size[0]

# Assuming emb_g is the original tensor
emb_g = torch.rand(original_size)

# Create new rows with random values
new_rows = torch.randn(num_new_rows, original_size[1], original_size[2])

# Concatenate the original tensor and new rows along the first dimension
new_tensor = torch.cat([emb_g, new_rows], dim=0)

# Check the size of the new tensor
print(new_tensor.size())  # Output: torch.Size([198, 192, 5]) due to concatenation along the first dimension


In [32]:
new_row.shape

torch.Size([6, 6, 5])

In [35]:
torch.cat([model.state_dict()[key], new_row], axis=2).shape

RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 198 but got size 6 for tensor number 1 in the list.

In [ ]:




mismatched_layers = []
for key, value in model_dict.items():
    if hasattr(model, 'module'):
        if key in model.module.state_dict() and value.size() != model.module.state_dict()[key].size():
        mismatched_layers.append(key)
    else:
        if key in model.state_dict() and value.size() != model.state_dict()[key].size():
        mismatched_layers.append(key)
        
print(mismatched_layers)

ignore_layers = ignore_layers + mismatched_layers
if len(ignore_layers) > 0:
    model_dict = {k: v for k, v in model_dict.items()
                    if k not in ignore_layers}
    if hasattr(model, 'module'):
        dummy_dict = model.module.state_dict()
        dummy_dict.update(model_dict)
    else:
        dummy_dict = model.state_dict()
        dummy_dict.update(model_dict)
    model_dict = dummy_dict

if hasattr(model, 'module'):
    model.module.load_state_dict(model_dict, strict=False)
else:
    model.load_state_dict(model_dict, strict=False)
return model

In [ ]:
generator = utils.warm_start_model(
            hps.train.warm_start_checkpoint, generator, hps.train.ignored_layer
        )

In [5]:
import os
with open("filelists/combine_audio_sid_text_test_filelist.txt", "r") as txt_file:
    lines = txt_file.readlines()
    for line in lines:
        audiopath = line.split("|")[0]
        if not os.path.isfile(audiopath):
            print(audiopath)

In [1]:
from text import text_to_sequence, cmudict
from text import cleaners
from unidecode import unidecode

In [2]:
tst_stn = "Hello my name is Pras"
tst_stn = "周末的我，只忙着陪你。"

tst_stn = " " + tst_stn.strip() + " "
text_norm = text_to_sequence(tst_stn.strip(), ['basic_cleaners'])

In [3]:
len(text_to_sequence(tst_stn.strip(), ['universal_cleaners']))

38

In [4]:
len(text_to_sequence(unidecode(tst_stn).strip(), ['basic_cleaners']))

37

In [10]:
unidecode(tst_stn)

' Zhou Mo De Wo ,Zhi Mang Zhao Pei Ni .  '

In [17]:
unidecode("喜んでもらえたならなによりです")

'Xi ndemoraetanarananiyoridesu'

In [ ]:
text = english_cleaners(tst_stn)

In [ ]:
def _clean_text(text, cleaner_names):
  for name in cleaner_names:
    cleaner = getattr(cleaners, name)
    if not cleaner:
      raise Exception('Unknown cleaner: %s' % name)
    text = cleaner(text)
  return text

In [6]:
text_norm

[60,
 52,
 11,
 61,
 46,
 11,
 45,
 58,
 38,
 51,
 11,
 39,
 42,
 46,
 11,
 47,
 46,
 51,
 44,
 11,
 45,
 58,
 38,
 51,
 11,
 62,
 46,
 51,
 44,
 11,
 51,
 46,
 11,
 7,
 11]